In [1]:
#Testing a model using all training data
import rascal
from rascal.representations import SphericalInvariants as SOAP

import ase
from ase import io
#from ase import atoms

import pickle
import os
import numpy as np
import matplotlib.pyplot as plt

import random

import soprano
from soprano.properties.nmr import *

import sklearn
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from numpy.linalg import lstsq

import pandas as pd

from pandas import DataFrame

In [2]:
def keys_grabber(category):
    keys = list(pickle.load(open('Data/' + category + '/uid_index.pkl','rb')).keys())
    for i in keys:
        if not os.path.exists('Data/'+category+'/' + str(i) + '.magres'):
            keys.remove(i)
    return keys

def descriptor(cut, smooth):
    HYPERS = {
    'soap_type': 'PowerSpectrum',
    'interaction_cutoff': cut,
    'max_radial': 2,
    'max_angular': 6,
    'gaussian_sigma_constant': 0.5,
    'gaussian_sigma_type': 'Constant',
    'cutoff_smooth_width': smooth,
    'radial_basis': 'GTO',
    'inversion_symmetry': True,
    'normalize' : True
    }
    soap = SOAP(**HYPERS)
    return soap


def puller(keys, soap, category):
    #reading in all structures and creating all spectrums.
    for i in keys:
        #print(i)
        if keys.index(i) == 0:
            structure = ase.io.read('Data/' + category +'/'+str(i)+'.magres')
            spectrum = soap.transform(structure).get_features(soap)
            full_spec = spectrum
            iso = MSIsotropy.get(structure)
        else:
            structure = ase.io.read('Data/' + category +'/'+str(i)+'.magres')
            spectrum = soap.transform(structure).get_features(soap)
            full_spec = np.concatenate((full_spec,spectrum),axis =0)
            iso = np.concatenate((iso,MSIsotropy.get(structure)),axis=0)
    return full_spec, iso


def splitter(tr_f, no_sparse, full_spec, iso):
    #Randomly choosing representative matrix
    ids = range(len(full_spec)) #list of all ids
    tr_id = random.sample(ids, int(tr_f*len(full_spec)))
    sp_id = random.sample(tr_id, no_sparse)

    tr_sp = full_spec[tr_id] #training spectrums
    sp_sp = full_spec[sp_id] #representative/sparse spectrums

    tr_ta = iso[tr_id] #training target
    
    te_id = list(ids)
    for i in tr_id:
        te_id.remove(i)
    te_sp = full_spec[te_id]
    te_ta = iso[te_id]
    
    return sp_sp, tr_sp, tr_ta, te_sp, te_ta 
    
    
def kerneller(to_kernel, sp_sp, ker_exp):
    kernel = (to_kernel@sp_sp.T)**ker_exp
    return kernel

In [8]:
def model_maker(cat, cut, smo_cut, sp_size, ker_exp, reg):
    for i in cat:
        if cat.index(i) == 0:
            keys = keys_grabber(i)
            soap = descriptor(cut, smo_cut)
            full_spec, iso = puller(keys, soap, i)
        else:
            keys = keys_grabber(i)
            soap = descriptor(cut, smo_cut)
            full_spec_t, iso_t = puller(keys, soap, i)
            full_spec = np.concatenate((full_spec, full_spec_t))
            iso = np.concatenate((iso, iso_t))
    sp_sp, tr_sp, tr_ta, te_sp, te_ta = splitter(1, sp_size, full_spec, iso)
    
    KNM = kerneller(tr_sp, sp_sp, ker_exp)
    KMM = kerneller(sp_sp, sp_sp, ker_exp)
    res = lstsq(KNM.T @ KNM + reg * KMM, KNM.T @ tr_ta, rcond=None)
    c = res[0]
    
    return c, sp_sp, iso

def predictor(cat, cut, smo_cut, ker_exp, reg, c, sp_sp):
#     keys = keys_grabber(cat)
    keys = [13]
    soap = descriptor(cut, smo_cut)
    full_spec, iso = puller(keys, soap, cat)
    KTM = kerneller(full_spec, sp_sp, ker_exp)
    pred = KTM @ c
    rmse = mean_squared_error(iso, pred, squared=False)
    return iso, pred, rmse

def species_splitter(target, predicted):
    ids_o = []
    ids_si = []

    for i in enumerate(target):
        if i[1] < 350:
            ids_o.append(i[0])
        else:
            ids_si.append(i[0])
            
    o_tar = target[ids_o]
    o_pred = predicted[ids_o]
    
    si_tar = target[ids_si]
    si_pred = predicted[ids_si]
    
    return o_tar, si_tar, o_pred, si_pred

In [4]:
#Params - nowusing same for all
cut = 3.5
smo_cut = 1
sp_size = 1000
ker_exp = 25
reg = 1*10**-8

In [12]:
rmses = []
counter =0
while counter <10:

    c, sp_sp, iso = model_maker(['HypoZeo', 'Rattle', 'AM0K', 'AM300K', 'MD_Distorted'],cut,smo_cut, sp_size, ker_exp,reg)
    tar, pred_r, rmse = predictor('Polymorphs', cut, smo_cut, ker_exp, reg, c, sp_sp)
    rmses.append(rmse)
    counter += 1
print(sum(rmses)/len(rmses))

2.8829022659688595


36513